<a href="https://colab.research.google.com/github/BhagwatiOracle/Currency-Converter/blob/main/CurrencyConverter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q langchain-openai langchain-core requests
import os
os.environ["OPENAI_API_KEY"] =''


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.7/437.7 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.5 MB/s eta 0:00:00


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [ ]:
from langchain_core.tools import InjectedToolArg
from typing import Annotated
@tool
def get_conversion_factor(base_currency:str, target_currency: str)->float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency

  """
  url=f'https://v6.exchangerate-api.com/v6/b1f945310995e0574671b906/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()


@tool
def convert(base_currency_value:int, conversion_rate: Annotated[float,InjectedToolArg])->float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """
  return base_currency_value*conversion_rate

In [ ]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1746662402,
 'time_last_update_utc': 'Thu, 08 May 2025 00:00:02 +0000',
 'time_next_update_unix': 1746748802,
 'time_next_update_utc': 'Fri, 09 May 2025 00:00:02 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 84.7697}

In [ ]:
convert.invoke({'base_currency_value':10, 'conversion_rate':85.76})

857.6

In [ ]:


llm = ChatOpenAI(model="Qwen/Qwen3-235B-A22B-fp8-tput",
    openai_api_base="https://api.together.xyz/v1")

In [ ]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [ ]:
message=[HumanMessage('What is the conversion factor between USD and INR and based on that can you convert 10 usd to inr')]

In [ ]:
ai_message=llm_with_tools.invoke(message)

In [ ]:
ai_message.tool_calls


[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'call_7xs81z824d2owddz07xqjc5q',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10},
  'id': 'call_2827lr2ynmdvqrud9qkhp3sl',
  'type': 'tool_call'}]

In [ ]:
message.append(ai_message)

In [ ]:
import json
for tool_call in ai_message.tool_calls:
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    print(tool_message1)

    conversion_rate=json.loads(tool_message1.content)['conversion_rate']

    message.append(tool_message1)

  if tool_call['name'] == 'convert':
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    message.append(tool_message2)



content='{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms_of_use": "https://www.exchangerate-api.com/terms", "time_last_update_unix": 1746662402, "time_last_update_utc": "Thu, 08 May 2025 00:00:02 +0000", "time_next_update_unix": 1746748802, "time_next_update_utc": "Fri, 09 May 2025 00:00:02 +0000", "base_code": "USD", "target_code": "INR", "conversion_rate": 84.7697}' name='get_conversion_factor' tool_call_id='call_7xs81z824d2owddz07xqjc5q'


In [ ]:
message

[HumanMessage(content='What is the conversion factor between USD and INR and based on that can you convert 10 usd to inr', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_7xs81z824d2owddz07xqjc5q', 'function': {'arguments': '{"base_currency":"USD","target_currency":"INR"}', 'name': 'get_conversion_factor'}, 'type': 'function', 'index': 0}, {'id': 'call_2827lr2ynmdvqrud9qkhp3sl', 'function': {'arguments': '{"base_currency_value":10}', 'name': 'convert'}, 'type': 'function', 'index': 1}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1252, 'prompt_tokens': 270, 'total_tokens': 1522, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen3-235B-A22B-fp8-tput', 'system_fingerprint': None, 'id': 'ntCk95J-6UHjtw-93c818100b63dd1c', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--349a449a-9ee8-43b9-8cb9-979ad8114849-0', tool_calls=[

In [ ]:
llm_with_tools.invoke(message).content

"<think>\nOkay, let's see. The user asked for the conversion factor between USD and INR and then wanted to convert 10 USD to INR.\n\nFirst, I need to call the get_conversion_factor function with base_currency as USD and target_currency as INR. The response from that gives the conversion rate, which is 84.7697. \n\nThen, using that conversion rate, I call the convert function with the base_currency_value of 10. Multiplying 10 by 84.7697 gives 847.697. \n\nSo, the final answer should present the conversion factor and the converted amount. I should format it clearly, maybe stating the factor first and then the converted value. Also, include the sources and terms as per the tool's response. Let me check if all the parameters are correctly used and if there's any missing info. Everything seems in order. Alright, time to put it all together in a user-friendly way.\n</think>\n\nThe conversion factor between USD and INR is **84.7697** (as of May 8, 2025, 00:00 UTC).  \n\nUsing this rate, **10 